In [19]:
import pandas as pd
import georinex as gr
import numpy as np
from data import Data

In [6]:
# initalize logging
from importlib import reload
import logging
reload(logging)
logging.basicConfig(level=logging.INFO)

## System dependencies
- install georinex, pandas, numpy, sklearn, yaml, and jupyter as python packages
- install dvc as a system package
- git clone this repo
- to enable the python-app.yml github action, use your github settings page and enable the workflow manually. This will allow you to train a new model with every commit to the `main` branch. 

## Data Preprocessing
Below we prepare the data as a np.array for both the `nav_df` and the `labels`. This can almost certainly be improved.


In [7]:
nav_file = '../2021.09.17_jamming/parsed-rinex/COM37_210917_110602_jamming.nav'
obs_file = '../2021.09.17_jamming/parsed-rinex/COM37_210917_110602_jamming.obs'
obs = gr.load(obs_file)
nav = gr.load(nav_file)
hdr_obs = gr.rinexheader(obs_file)
hdr_nav = gr.rinexheader(nav_file)
obs_df = obs.to_dataframe()
obs_df.fillna(value = 0, axis = 1)
obs_df = obs_df.sort_index()
labels = np.zeros(len(obs_df))
tmp = int(12350/5)
tmp2 = tmp * 2
labels[tmp2: tmp2+tmp] = np.ones(tmp)
assert set(labels) == set([0,1])
obs_df = np.nan_to_num(obs_df)

## This code snippet is inside the data.py file, accessible by calling Data(dataset= 'rinex-obs')

C:\tools\Anaconda3\envs\survey\lib\site-packages\numpy\core\fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\tools\Anaconda3\envs\survey\lib\site-packages\numpy\core\_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\tools\Anaconda3\envs\survey\lib\site-packages\numpy\core\fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\tools\Anaconda3\envs\survey\lib\site-packages\numpy\core\_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\tools\Anaconda3\envs\survey\lib\site-packages\numpy\core\fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\tools\Anaconda3\envs\survey\lib\site-packages\numpy\core\_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = 

## Data Preparation
specified in configs/prepare.yml and can be changed by command line or function arguments. Supports datasets outlined in data.py as well as arbitrary csvs, assuming the last column is the target vector.

## Initial Model Training

because each layer performs a grid search on the best hyper-parameter and object combination specified in the .yml file, we need to train an initial model so that we can make objective measurements as we add more layers.

## Data Preprocessing

specified in `configs/preprocess.yml` and can be changed with command-line or functional arguments. Supports any sklearn preprocessor from [this documentation](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing). Below we declare the paths for our configuration file and results folder.

## Data Featurizing

specified in `configs/preprocess.yml` and can be changed with command-line or functional arguments. Supports any sklearn preprocessor from [this documentation](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing). Below we declare the paths for our configuration file and results folder.


## Evaluation of the data
TODO:
Once we know what our best model is, we can use it to predict on a new set of data using the evaluate.py script. Parameters for scoring can be specified in evaluate.yml. Please note that metrics that require parameters other than `y_test` or `y_pred` at runtime must first be made into a scorer callable using the sklearn make_scorer() function. 

## Defense
TODO

## Attack
TODO


## DVC

This repo uses a dvc pipeline to store the data pipeline in a git repository. You can see the pipeline in `dvc.yaml` which specifies input and output params for each layer, as well as what script to run. After performing any edits run: ```dvc repro``` to reproduce the experiment. Please note it will only re-run pipeline components that have not changed, so it is imperative that all dependencies are explicit. When that is done, view the metrics with ```dvc metrics show```. You can also generate plots automatically or from the command line with ```dvc plots modify```. You can run the commands from inside a jupyter notebook by preprending system commands with a '!' as demonstated below.

In [4]:
 # shows the dag of the pipeline
!dvc dag

  +---------+  
  | prepare |  
  +---------+  
       *       
       *       
       *       
  +-------+    
  | train |    
  +-------+    
       *       
       *       
       *       
+------------+ 
| preprocess | 
+------------+ 
       *       
       *       
       *       
+-----------+  
| featurize |  
+-----------+  


In [1]:
# reproduces the pipeline on your local machine
!dvc repro > repro.log

'tail' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
# Displays the metrics specified in the experiment object, set by (TODO) `evaluate.py`
! dvc metrics diff

Path                               Metric             HEAD        workspace      Change
data\best_preprocess\results.json  Accuracy           0.95337     0.80246        -0.15091
data\best_preprocess\results.json  Balanced Accuracy  0.88618     0.51534        -0.37084
data\best_preprocess\results.json  F1                 0.9512      0.72148        -0.22972
data\best_preprocess\results.json  Precision          0.95483     0.81568        -0.13915
data\best_preprocess\results.json  Recall             0.95337     0.80246        -0.15091
data\best_preprocess\results.json  fit_time           703125000   1177656250000  1176953125000
data\best_preprocess\results.json  pred_time          31250000    859375000      828125000
data\best_train\results.json       Accuracy           0.95758     0.79696        -0.16062
data\best_train\results.json       Balanced Accuracy  0.89254     0.5            -0.39254
data\best_train\results.json       F1                 0.95559     0.70691        -0.24868
data\b

### 